In [1]:
from __future__ import print_function

# $example on$
from pyspark.ml.feature import StringIndexer
from pyspark import SparkContext
from pyspark.sql import SQLContext


sql_sc = SQLContext(sc)

In [8]:
fname = '../data/simple_ad.txt'
adv_data = []
with open(fname, 'r') as f:
    for line in f:
        if not line.startswith('#'):
            adv_data.append([line.split('\n')[0]])
adv_data = adv_data[1:]
print(adv_data[:5])

[['101com.com'], ['101order.com'], ['123found.com'], ['140proof.com'], ['180hits.de']]


In [ ]:
# First start the analysis for a sample 1 hour data, we can easily expand this to multiple hours. 
fname = '/tigress/arpitg/dns_data/tshark_20170207_20170212_consolidated/split_hour_00118_20170211215900.csv'
#fname = 'data/tshark_20170207_20170212_consolidated/split_hour_00118_20170211215900.csv'


import glob
dns_fnames = glob.glob("../data/tshark_20170207_20170212_consolidated/*.csv")

dns_data = []
for dns_fname in dns_fnames[:24]:
    print("Processing file", dns_fname)
    with open(fname, 'r') as f:
        for line in f:
            tmp = line.split('\n')[0].split(',')
            dns_data.append(tuple(tmp))

Processing file ../data/tshark_20170207_20170212_consolidated/split_hour_00118_20170211215900.csv


In [ ]:
print (dns_data[:2],len(dns_data))

In [ ]:
dns_data_df = sql_sc.createDataFrame(dns_data,["ts", "xx", "srcIP", "dstIP", "qname", "qtype", "macAddr", "subnet", 
                                               "host_type", "host_name", "system_type", "OS", "nic_manuf"])

In [ ]:
adv_data_df = sql_sc.createDataFrame(adv_data,["qname"])

In [ ]:
print(adv_data[:5])

In [ ]:
dns_data_joined = dns_data_df.join(adv_data_df, "qname", "inner")
affected_devices = dns_data_joined.select("srcIP").distinct()

In [ ]:
# Get fraction of DNS responses that were know advertisement links

total_responses_for_affected_devices = (dns_data_df.join(affected_devices, "srcIP", "inner")
                              .select(["srcIP", "qname"])
                              .groupBy("srcIP").count()
                              .withColumnRenamed('count', 'total_count')
                             )

adv_responses_for_affected_devices = (dns_data_joined
                                      .select(["srcIP", "qname"])
                                      .groupBy("srcIP")
                                      .count()
                                      .withColumnRenamed('count', 'adv_count')
                                     )

fraction_adv_responses_for_affected_devices = (total_responses_for_affected_devices
                                               .join(adv_responses_for_affected_devices, "srcIP", "outer")
                                               .rdd.map(lambda p: (p.srcIP, 100*float(p.adv_count)/p.total_count))
                                               .toDF(["srcIP", "percentage_adv"])
                                              )

fraction_adv_responses_for_affected_devices.show()

In [ ]:
# 10.8.213.252: an IP with very high advertisement traffic


In [ ]:
print(dns_data_df.filter(dns_data_df["srcIP"] == "10.8.213.252").select("ts").rdd.map(lambda p: str(p.ts)).collect())